# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f948c604b20>
├── 'a' --> tensor([[0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490]])
└── 'x' --> <FastTreeValue 0x7f948c6048e0>
    └── 'c' --> tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                        [ 1.0968, -0.7212, -0.6190,  0.2492],
                        [-0.3020, -0.3550, -1.5356,  1.5305]])

In [4]:
t.a

tensor([[0.1731, 1.2431, 1.0552],
        [1.5882, 0.4496, 0.6490]])

In [5]:
%timeit t.a

65.8 ns ± 0.0434 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f948c604b20>
├── 'a' --> tensor([[ 1.4979, -0.1005, -0.2777],
│                   [-0.3962,  1.7643,  1.6042]])
└── 'x' --> <FastTreeValue 0x7f948c6048e0>
    └── 'c' --> tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                        [ 1.0968, -0.7212, -0.6190,  0.2492],
                        [-0.3020, -0.3550, -1.5356,  1.5305]])

In [7]:
%timeit t.a = new_value

67 ns ± 0.0542 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.1731, 1.2431, 1.0552],
               [1.5882, 0.4496, 0.6490]]),
    x: Batch(
           c: tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305]]),
       ),
)

In [10]:
b.a

tensor([[0.1731, 1.2431, 1.0552],
        [1.5882, 0.4496, 0.6490]])

In [11]:
%timeit b.a

54.7 ns ± 0.0978 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.8944,  1.2695,  0.0452],
               [-1.0314, -1.7845, -0.4963]]),
    x: Batch(
           c: tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.143 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 0.193 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 33.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 410 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 721 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f93e6e99f10>
├── 'a' --> tensor([[[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]],
│           
│                   [[0.1731, 1.2431, 1.0552],
│                    [1.5882, 0.4496, 0.6490]]])
└── 'x' --> <FastTreeValue 0x7f93e08a1bb0>
    └── 'c' --> tensor([[[ 1.0403,  0.9743,  1.6760, -0.5513],
           

In [21]:
%timeit t_stack(trees)

31.2 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f948c604430>
├── 'a' --> tensor([[0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490],
│                   [0.1731, 1.2431, 1.0552],
│                   [1.5882, 0.4496, 0.6490]])
└── 'x' --> <FastTreeValue 0x7f948c604c40>
    └── 'c' --> tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                        [ 1.0968, -0.7212, -0.6190,  0.2492],
                        [-0.3020, -0.3550, -1.5356,  1.5305],
   

In [23]:
%timeit t_cat(trees)

29.7 µs ± 66.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 91 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.0403,  0.9743,  1.6760, -0.5513],
                       [ 1.0968, -0.7212, -0.6190,  0.2492],
                       [-0.3020, -0.3550, -1.5356,  1.5305]],
              
                      [[ 1.0403,  0.9743,  1.6760, -0.5513],
                       [ 1.0968, -0.7212, -0.6190,  0.2492],
                       [-0.3020, -0.3550, -1.5356,  1.5305]],
              
                      [[ 1.0403,  0.9743,  1.6760, -0.5513],
                       [ 1.0968, -0.7212, -0.6190,  0.2492],
                       [-0.3020, -0.3550, -1.5356,  1.5305]],
              
                      [[ 1.0403,  0.9743,  1.6760, -0.5513],
                       [ 1.0968, -0.7212, -0.6190,  0.2492],
                       [-0.3020, -0.3550, -1.5356,  1.5305]],
              
                      [[ 1.0403,  0.9743,  1.6760, -0.5513],
                       [ 1.0968, -0.7212, -0.6190,  0.2492],
                       [-0.3020, -0.3550, -1.5356,  1.5305]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305],
                      [ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305],
                      [ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305],
                      [ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305],
                      [ 1.0403,  0.9743,  1.6760, -0.5513],
                      [ 1.0968, -0.7212, -0.6190,  0.2492],
                      [-0.3020, -0.3550, -1.5356,  1.5305],
                      [ 1.0403,  0.9743,  1.6760, -0.5513],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
